In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import time
import datetime
import re
import cPickle as pickle

from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error

In [188]:
### MAKE PICKLES OF NEW DATAFRAMES
infile_skills_vectorized = 'data/_skills_vectorized_df.pkl'
infile_profiles = 'data/profiles_df.pkl'
infile_jobs = 'data/jobs_df.pkl'
infile_hr_paid_jobs = 'data/hr_paid_jobs_df.pkl'
infile_skills = 'data/skills_df.pkl'

In [189]:
#  THESE DATA ARE FROM THE (PROVIDER) PROFILES DATA SET
#  ALSO GOING TO TAP THE RICHER JOBS PROFILES DATA SET BELOW
jobs_df = pickle.load(open(infile_jobs, 'rb'))

profiles_df = pickle.load(open(infile_profiles, 'rb'))
skills_df = pickle.load(open(infile_skills, 'rb'))

In [113]:
profiles_df.T.head(35)

,0,1,2,3,4,5,6,7,8,9,...,88686,88687,88688,88689,88690,88691,88692,88693,88694,88695
dev_bill_rate,15,12,44.44,24.44,15,16.67,30,11.11,10,5.56,...,22.22,4,6.67,13.89,10,16.67,11.11,35,3,30
dev_blurb,"Over the more than 6 years, I have developed a...","I am a web application developer, having 2 yea...",Extensive expertise in clinical research publi...,I'm a WEB-developer with more than 5 years exp...,Specializes in developing Photoshop and After ...,,skype : orange_project_kh\nemail: contact@...,DTL Solutions is the software solutions provid...,I am an upcoming college graduate! I will rece...,1) Shopping Card Management System using C# a...,...,I'm a software engineer focused on backend and...,I am working as a officer at Checkpoint syst...,To enhance my journalistic prowess through onl...,I'm a highly motivated person with a strong ba...,"I like building cool web applications, and I o...",I configure and run the base station WiMAX on ...,I am looking to help in software solutions . w...,I have extensive experience using Python to so...,"Looking for work in proofreading, editing or c...",I have 8 Years of Profession Experience and i ...
dev_country,India,India,United States,Ukraine,Sri Lanka,Germany,Ukraine,Vietnam,United States,Bangladesh,...,Russia,Bangladesh,Philippines,Russia,India,Russia,Egypt,United States,India,India
dev_eng_skill,4,5,5,5,,5,4,4,5,,...,3,4,4,4,5,3,5,5,4,5
dev_portrait,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,https://www.odesk.com/att/~~mOJyJwUvL-dOchJGOo...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...
dev_profile_title,LAMP developer with more than 6 years,Web Developer / Designer,"English/French Writer/Translator,Scientific Di...",PHP/Zend Framework/MySql/MongoDB/SPHINX/jQuery...,"After Effects, Photoshop - Scripting And Plugi...",Development of reliable and flexible software(...,"Fullstack web developer, e-commerce developing...",Small IT solution company provides Development...,Upbeat Enthusiastic Professional Writer,Bsc in Computer Science and Engineering,...,Software Developer,IT Professional,Statistician|News&Sports Writer|InDesign|Pagem...,Experienced developer and scientific assistant,An expert web application developer with Djang...,Software Developer,Ahmed Eshra,Python & ArcGIS Programmer,,Project Lead
dev_recno_ciphertext,~01cae3810dc6d2bbbc,~01d24ced467385214d,~01953fb4fa2ceb67dc,~01a9ab111f973ae055,~016dcec1087cba565c,~019e83a0dfa04c0e8b,~01863825efc778f975,~01614e82e3c5a0ccca,~0122260b02b0c56563,~01a3583a33f5146e8e,...,~01dfd63f31c5228d16,~018aaab6cfdc3fb96c,~01360017155d421ae3,~0169007d9fa447a06b,~014920c811cecebbc4,~0148cf1eca9907db89,~01baacbe6b9e3d8c30,~013659f86de224bdf9,~01b9ca1c9967daf519,~01a0cdfa47888e70f7
dev_timezone,"UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+02:00 Athens, Helsinki, Istanbul","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC+02:00 Eastern Europe,"UTC+07:00 Omsk, Novosibirsk",UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+06:00 Almaty, Dhaka",...,"UTC+07:00 Omsk, Novosibirsk","UTC+06:00 Almaty, Dhaka","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei","UTC+03:00 Moscow, St. Petersburg, Volg

In [114]:
skills_df.T.head(35)

,0,1,2,3,4,5,6,7,8,9,...,691868,691869,691870,691871,691872,691873,691874,691875,691876,691877
dev_recno_ciphertext,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,~01cae3810dc6d2bbbc,...,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7,~01a0cdfa47888e70f7
skl_name,ajax,api-development,curl,javascript,magento,mysql,php,smtp,xml,xml-rpc,...,apache-struts,hibernate,java,java-servlets-development,javascript,jpa,jsp,oracle-database,richfaces,spring-framework


# Investigate jobs profiles data set

In [115]:
def json_prep(in_file):
    '''
    Create pandas dataframe from text file of JSON strings
    '''
    # read the entire file `ainto a python array
    with open(in_file, 'rb') as f:
        data = f.readlines()

    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # each element of 'data' is an individual JSON object.
    # i want to convert it into an *array* of JSON objects
    # which, in and of itself, is one large JSON object
    # basically... add square brackets to the beginning
    # and end, and have all the individual business JSON objects
    # separated by a comma
    data_json_str = "[" + ','.join(data) + "]"

    data_list_of_dicts = json.loads(data_json_str)

    out_df = pd.read_json(data_json_str)
    return out_df

In [116]:
def grab_data(datalist, creature, id_, list_of_keys):
    if isinstance(creature, basestring):
        pass
    elif isinstance(creature, dict):
        tuple_to_add = tuple([id_] + [creature.get(key, '') for key in list_of_keys])
        datalist.append(tuple_to_add)
    elif isinstance(creature, list):
        for creature_item in creature:
            tuple_to_add = tuple([id_] + [creature_item.get(key, '') for key in list_of_keys])
            datalist.append(tuple_to_add)

In [117]:
def triage(dict_, level=0):
    '''
    Visualize JSON data
    "Pretty Print" data, showing nesting via indentation
    '''
    if not(isinstance(dict_, float) or isinstance(dict_, basestring)):
        print type(dict_)
        for key, val in dict_.iteritems():
            print ' '*level*2, key
            if isinstance(val, dict):
                triage(val, level=level+1)
            elif isinstance(val, list):
                for index, item in enumerate(val):
                    print ' '*level*2, 'item', index
                    triage(item, level=level+1)
            elif isinstance(val, basestring):
                print ' '*(level+1)*2, '=', val

In [193]:
jobs_profiles_df = json_prep('data/jobs_da_0.txt')

In [119]:
jobs_profiles_df.T.head(40)

,0,1,2,3,4,5,6,7,8,9,...,7872,7873,7874,7875,7876,7877,7878,7879,7880,7881
amount,30,5,,,150,250.00,8,,400.00,,...,20,0.00,,,20,400.00,,,,
assignment_info,"{u'info': {u'end_data': u'1338768000000', u'ci...","{u'info': {u'end_data': u'1318982400000', u'ci...","{u'info': {u'total_charge': u'498.33', u'end_d...","{u'info': [{u'total_charge': u'0', u'end_data'...","{u'info': {u'end_data': u'1376006400000', u'ci...","{u'info': {u'end_data': u'1416355200000', u'ci...","{u'info': {u'end_data': u'1389657600000', u'ci...","{u'info': {u'end_data': u'1318809600000', u'ci...","{u'info': [{u'total_charge': u'0', u'end_data'...","{u'info': {u'end_data': u'1436313600000', u'ci...",...,"{u'info': {u'end_data': u'1334966400000', u'ci...","{u'info': [{u'end_data': u'1451347200000', u'c...","{u'info': [{u'end_data': u'1359590400000', u'c...","{u'info': {u'end_data': u'1349740800000', u'ci...","{u'info': {u'end_data': u'1275523200000', u'ci...","{u'info': {u'end_data': u'1449792000000', u'ci...","{u'info': {u'end_data': u'1354233600000', u'ci...","{u'info': {u'end_data': u'1342915200000', u'ci...","{u'info': {u'end_data': u'1282089600000', u'ci...","{u'info': {u'end_data': u'1356393600000', u'ci..."
assignments,"{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'3.33', ...","{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'04/2016', u'as_r...","{u'assignment': [{u'as_to': u'12/2013', u'as_r...","{u'assignment': {u'as_total_hours': u'0', u'as...","{u'assignment': [{u'as_to': u'Present', u'as_r...",...,"{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'52', u'...","{u'assignment': [{u'as_to': u'02/2016', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': {u'as_to': u'11/2012', u'as_ra...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'01/2011', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r..."
buyer,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC (Coordinated Universal ...,{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+10:00 Brisbane', u'op_t...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,...,{u'op_timezone': u'UTC-06:00 Central Time (US ...,"{u'op_timezone': u'UTC+05:00 Islamabad, Karach...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+10:00 Sydney, Melbourne...",{u'op_timezone': u'UTC (Coordinated Universal ...,"{u'op_timezone': u'UTC+08:00 Hong Kong SAR, Pe...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm..."
candidates,{u'candidate': {u'ciphertext': u'~01ca3b6ff3e6...,{u'candidate': {u'ciphertext': u'~01eef64ef43a...,{u'candidate': [{u'ciphertext': u'~01f1c24717c...,{u'candidate': [{u'ciphertext': u'~01cde4ef409...,{u'candidate': [{u'ciphertext': u'~01e546ffa0d...,{u'candidate': [{u'ciphertext': u'~01e02cdeeb0...,{u'candidate': [{u'ciphertext': u'~01ee1be0350...,{u'candidate': [{u'ciphertext': u'~010973123f7...,{u'candidate': [{u'ciphertext': u'~018c64192f2...,{u'candidate': [{u'ciphertext': u'~011a4fcf20c...,...,{u'candidate': {u'ciphertext': u'~01f120be2a0d...,{u'candidate': {u'ciphertext': u'~01e495c52e01...,{u'candidate': [{u'ciphertext': u'~015a583df5a...,{u'candidate': [{u'ciphertext': u'~010d9ec6757...,,,{u

### Drop evidently unuseful columns

### Data leak

In [120]:
jobs_profiles_df.drop([ \
                       'amount', \
                       'engagement_weeks', \
                       'op_other_jobs', \
                       'ui_opening_status', \
                      ], axis=1, inplace=True)


### Irrelevant

In [121]:
jobs_profiles_df.drop([ \
                        'interviewees_total_active', \
                        'op_additional_questions', \
                        'op_attached_doc', \
                        'op_cny_upm_verified', \
                        'op_is_cover_letter_required', \
                        'op_pref_group_id', \
                        'op_pref_group_logo', \
                        'op_pref_group_name' \
                      ], axis=1, inplace=True)


### Deprecated

In [122]:
jobs_profiles_df.drop([ \
                        'job_category_level_one', \
                        'job_category_level_two'
                      ], axis=1, inplace=True)


### Unsure and unlikely

In [123]:
jobs_profiles_df.drop([ \
                        'op_contractor_tier', \
                        'op_engagement', \
                        'op_high_hourly_rate_all', \
                        'op_low_hourly_rate_all', \
                        'op_pref_english_skill', \
                        'op_pref_fb_score', \
                        'op_pref_has_portfolio', \
                        'op_pref_hourly_rate_max', \
                        'op_pref_hourly_rate_min', \
                        'op_pref_location', \
                        'op_pref_odesk_hours', \
                        'op_tot_feedback' \
                      ], axis=1, inplace=True)


In [124]:
jobs_profiles_df.T.head(20)

,0,1,2,3,4,5,6,7,8,9,...,7872,7873,7874,7875,7876,7877,7878,7879,7880,7881
assignment_info,"{u'info': {u'end_data': u'1338768000000', u'ci...","{u'info': {u'end_data': u'1318982400000', u'ci...","{u'info': {u'total_charge': u'498.33', u'end_d...","{u'info': [{u'total_charge': u'0', u'end_data'...","{u'info': {u'end_data': u'1376006400000', u'ci...","{u'info': {u'end_data': u'1416355200000', u'ci...","{u'info': {u'end_data': u'1389657600000', u'ci...","{u'info': {u'end_data': u'1318809600000', u'ci...","{u'info': [{u'total_charge': u'0', u'end_data'...","{u'info': {u'end_data': u'1436313600000', u'ci...",...,"{u'info': {u'end_data': u'1334966400000', u'ci...","{u'info': [{u'end_data': u'1451347200000', u'c...","{u'info': [{u'end_data': u'1359590400000', u'c...","{u'info': {u'end_data': u'1349740800000', u'ci...","{u'info': {u'end_data': u'1275523200000', u'ci...","{u'info': {u'end_data': u'1449792000000', u'ci...","{u'info': {u'end_data': u'1354233600000', u'ci...","{u'info': {u'end_data': u'1342915200000', u'ci...","{u'info': {u'end_data': u'1282089600000', u'ci...","{u'info': {u'end_data': u'1356393600000', u'ci..."
assignments,"{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'3.33', ...","{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'04/2016', u'as_r...","{u'assignment': [{u'as_to': u'12/2013', u'as_r...","{u'assignment': {u'as_total_hours': u'0', u'as...","{u'assignment': [{u'as_to': u'Present', u'as_r...",...,"{u'assignment': [{u'as_total_hours': u'0', u'a...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_total_hours': u'52', u'...","{u'assignment': [{u'as_to': u'02/2016', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': {u'as_to': u'11/2012', u'as_ra...","{u'assignment': [{u'as_to': u'Present', u'as_r...","{u'assignment': [{u'as_to': u'01/2011', u'as_r...","{u'assignment': [{u'as_to': u'Present', u'as_r..."
buyer,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC (Coordinated Universal ...,{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+10:00 Brisbane', u'op_t...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,...,{u'op_timezone': u'UTC-06:00 Central Time (US ...,"{u'op_timezone': u'UTC+05:00 Islamabad, Karach...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm...",{u'op_timezone': u'UTC-05:00 Eastern Time (US ...,"{u'op_timezone': u'UTC+10:00 Sydney, Melbourne...",{u'op_timezone': u'UTC (Coordinated Universal ...,"{u'op_timezone': u'UTC+08:00 Hong Kong SAR, Pe...",{u'op_timezone': u'UTC-08:00 Pacific Time (US ...,"{u'op_timezone': u'UTC+01:00 Berlin, Stockholm..."
candidates,{u'candidate': {u'ciphertext': u'~01ca3b6ff3e6...,{u'candidate': {u'ciphertext': u'~01eef64ef43a...,{u'candidate': [{u'ciphertext': u'~01f1c24717c...,{u'candidate': [{u'ciphertext': u'~01cde4ef409...,{u'candidate': [{u'ciphertext': u'~01e546ffa0d...,{u'candidate': [{u'ciphertext': u'~01e02cdeeb0...,{u'candidate': [{u'ciphertext': u'~01ee1be0350...,{u'candidate': [{u'ciphertext': u'~010973123f7...,{u'candidate': [{u'ciphertext': u'~018c64192f2...,{u'candidate': [{u'ciphertext': u'~011a4fcf20c...,...,{u'candidate': {u'ciphertext': u'~01f120be2a0d...,{u'candidate': {u'ciphertext': u'~01e495c52e01...,{u'candidate': [{u'ciphertext': u'~015a583df5a...,{u'candidate': [{u'ciphertext': u'~010d9ec6757...,,,{u'candidate': [{u'ciphertext': u'~01f9d6c47b2...,{u'candidate': [

# Create derived variables

In [125]:
def convert_millisecond_epoch_datetime(str):
    if str <> '':
        return datetime.datetime.fromtimestamp(int(str)/1000)
    else:
        return np.datetime64('nat')

In [126]:
def convert_epoch_datetime(str):
    if str <> '':
        return datetime.datetime.fromtimestamp(int(str))
    else:
        return np.datetime64('nat')

### `assignment_info` : contains info on Providers who obtained the Assignment

In [127]:
assi_tuples = list()
key_list = ['ciphertext_developer_recno', 'start_date', 'end_data']
for index, assi_tuple in \
  enumerate(jobs_profiles_df[['ciphertext', 'assignment_info']].itertuples(index=False)):
    profile_id = assi_tuple[0]
    assi_info = assi_tuple[1]
    if 'info' in assi_info:
        grab_data(assi_tuples, assi_info['info'], profile_id, key_list)

In [128]:
assi_df = pd.DataFrame(assi_tuples)
assi_df.columns = ['ciphertext'] + key_list

In [129]:
assi_df['start_datetime'] = assi_df['start_date'].apply(convert_millisecond_epoch_datetime)
assi_df['end_datetime'] = assi_df['end_data'].apply(convert_millisecond_epoch_datetime)
assi_df.drop(['start_date', 'end_data'], axis=1, inplace=True)

### `assignments` is a history of jobs offered by the Buyer

In [130]:
# for index, assi_tuple in \
#   enumerate(jobs_profiles_df[['ciphertext', 'assignments']].itertuples(index=False)):
#     profile_id = assi_tuple[0]
#     assi_info = assi_tuple[1]
#     if isinstance(assi_info, dict):
#         triage(assi_info)
#         print
#     if index > 1: break

### `buyer`

#### Note no buyer id available in `buyer` key

In [131]:
buyers_tuples = list()
key_list = ['op_timezone', 'op_country', 'op_city', 'op_contract_date']
for index, buyer_tuple in \
  enumerate(jobs_profiles_df[['ciphertext', 'buyer']].itertuples(index=False)):
    profile_id = buyer_tuple[0]
    buyer_info = buyer_tuple[1]
    if isinstance(buyer_info, dict):
        grab_data(buyers_tuples, buyer_info, profile_id, key_list)

In [132]:
buyer_df = pd.DataFrame(buyers_tuples)
buyer_df.columns = ['ciphertext'] + key_list

### `candidate`

In [133]:
cand_tuples = list()
key_list = ['ciphertext', 'create_date_ts']

for index, cand_tuple in \
  enumerate(jobs_profiles_df[['ciphertext', 'candidates']].itertuples(index=False)):
    profile_id = cand_tuple[0]
    cand_info = cand_tuple[1]
    if 'candidate' in cand_info:
        grab_data(cand_tuples, cand_info['candidate'], profile_id, key_list)

In [134]:
cand_df = pd.DataFrame(cand_tuples)
cand_df.columns = ['ciphertext', 'cand_ciphertext', 'create_date_ts']

In [135]:
cand_df['create_datetime'] = cand_df['create_date_ts'].apply(convert_epoch_datetime)
cand_df.drop(['create_date_ts'], axis=1, inplace=True)

### `op_ctime`

In [136]:
jobs_profiles_df['job_post_date'] = \
  jobs_profiles_df['op_ctime'].apply(convert_millisecond_epoch_datetime)
jobs_profiles_df.drop(['op_ctime'], axis=1, inplace=True)

### `op_description`

In [137]:
def stripped(str):
    return ' '.join(str.split())

In [138]:
jobs_profiles_df['op_description_stripped'] = jobs_profiles_df['op_description'].apply(stripped)
jobs_profiles_df.drop(['op_description'], axis=1, inplace=True)

### `op_job_category_v2`: Get category group (larger in size than category)

In [139]:
categ_tuples = list()
key_list = ['name']

for index, cat2_tuple in \
  enumerate(jobs_profiles_df[['ciphertext', 'op_job_category_v2']].itertuples(index=False)):
    profile_id = cat2_tuple[0]
    cat2_info = cat2_tuple[1]
    if isinstance(cat2_info, dict):
        if 'op_job_category_v' in cat2_info:
#             print cat2_info['op_job_category_v']
            if 'groups' in cat2_info['op_job_category_v']:
                groups_info = cat2_info['op_job_category_v']['groups']
                if 'group' in groups_info:
                    categ_tuples.append((profile_id, groups_info['group']['name']))

In [140]:
categ_df = pd.DataFrame(categ_tuples)
categ_df.columns = ['ciphertext', 'category_group']

In [141]:
jobs_profiles_df.drop(['op_job_category_v2'], axis=1, inplace=True)

### `op_required_skills`

In [142]:
def multiple_replace(dict, text):
    # Make one-letter skills longer
    if text == 'c':
        longtext = 'cprog'
    elif text == 'r':
        longtext = 'rprog'
    elif text == 's':
        longtext = 'sprog'
    else:
        longtext = text

    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], longtext)

substitution_dict = { \
    '+': 'plus', \
    '#': 'sharp', \
    '/': 'slash', \
    '-': 'dash', \
    '.': 'dot' \
}

def contiguous_word(skill):
    return multiple_replace(substitution_dict, skill)

In [143]:
skills_tuples = list()
key_list = ['skill']

for index, jobs_tuple in \
  enumerate(jobs_profiles_df[['ciphertext', 'op_required_skills']].itertuples(index=False)):
    profile_id = jobs_tuple[0]
    skill_info = jobs_tuple[1]
    if isinstance(skill_info, dict):
        if 'op_required_skill' in skill_info:
            grab_data(skills_tuples, skill_info['op_required_skill'], profile_id, key_list)

In [144]:
skills_set_list = list()
key_list = ['job_skill_set']
start_tuple_number = 0
end_tuple_number = len(skills_tuples) - 1

for index, skills_tuple in enumerate(skills_tuples):
    if index == start_tuple_number:
        curr_ciphertext = skills_tuple[0]
        skills_set = set( {contiguous_word(skills_tuple[1])} )
    elif curr_ciphertext == skills_tuple[0]:
        skills_set.add(contiguous_word(skills_tuple[1]))
    elif curr_ciphertext <> skills_tuple[0]:
        skills_set_list.append((curr_ciphertext, skills_set.copy()))
        skills_set = set( {contiguous_word(skills_tuple[1])} )
        curr_ciphertext = skills_tuple[0]

    if index == end_tuple_number:
        skills_set_list.append((curr_ciphertext, skills_set))

In [145]:
print len(skills_tuples)
print len(skills_set_list)

16473
5725


In [146]:
req_skills_df = pd.DataFrame(skills_set_list)
req_skills_df.columns = ['ciphertext'] + key_list

In [147]:
jobs_profiles_df.drop(['op_required_skills'], axis=1, inplace=True)

In [148]:
df_list = [ \
assi_df, \
buyer_df, \
cand_df, \
categ_df, \
req_skills_df \
]

for df in df_list:
    print df.info()
    print

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11382 entries, 0 to 11381
Data columns (total 4 columns):
ciphertext                    11382 non-null object
ciphertext_developer_recno    11382 non-null object
start_datetime                11382 non-null datetime64[ns]
end_datetime                  10142 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 444.6+ KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7882 entries, 0 to 7881
Data columns (total 5 columns):
ciphertext          7882 non-null object
op_timezone         7882 non-null object
op_country          7882 non-null object
op_city             7882 non-null object
op_contract_date    7882 non-null object
dtypes: object(5)
memory usage: 369.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146106 entries, 0 to 146105
Data columns (total 3 columns):
ciphertext         146106 non-null object
cand_ciphertext    146106 non-null object
create_datetime    146106 non-null datetime64[ns

In [149]:
assi_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,11372,11373,11374,11375,11376,11377,11378,11379,11380,11381
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037164e7e1613e2f,~01037164e7e1613e2f,~01037313aee93f572b,~010373177cb75fac67,~01037460b8b2e770bb,~010374b5de700160df,...,~01274cff9a4becab2e,~01274d1cb929fb7d32,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
ciphertext_developer_recno,~01ca3b6ff3e632f6ab,~01eef64ef43a757c51,~0140a8c1b287ababb9,~0172e72850ac36a641,~01daf57550023b5bbe,~01174ba84e44af17a5,~012ac81a79539b8972,~01a15cfa577cb31a5d,~017129c3d0ac9e32e2,~010973123f73f21e24,...,~01ee6ef35b5fe3a0f2,~0103353dd45da96b07,~010535fd344e7b9ca3,~0163ceb94c37a856e2,~0151206f7fbb479350,~014606c48dfaaf5f8f,~01d3f2d9207ca51d55,~01820a3866e9ecb989,~011fbff93e5f5f1117,~014e77cc7d6c4d44b7
start_datetime,2012-02-18 16:00:00,2011-10-17 17:00:00,2015-05-05 17:00:00,2015-11-02 16:00:00,2015-11-02 16:00:00,2015-11-01 16:00:00,2013-06-12 17:00:00,2014-09-13 17:00:00,2014-01-13 16:00:00,2011-08-09 17:00:00,...,2015-12-02 16:00:00,2012-10-25 17:00:00,2012-10-24 17:00:00,2010-04-30 17:00:00,2010-06-02 17:00:00,2015-12-06 16:00:00,2012-11-06 16:00:00,2012-03-17 17:00:00,2010-08-14 17:00:00,2012-06-10 17:00:00
end_datetime,2012-06-03 17:00:00,2011-10-18 17:00:00,NaT,2016-01-25 16:00:00,NaT,NaT,2013-08-08 17:00:00,2014-11-18 16:00:00,2014-01-13 16:00:00,2011-10-16 17:00:00,...,2016-01-02 16:00:00,2013-01-30 16:00:00,2013-01-26 16:00:00,2012-10-08 17:00:00,2010-06-02 17:00:00,2015-12-10 16:00:00,2012-11-29 16:00:00,2012-07-21 17:00:00,2010-08-17 17:00:00,2012-12-24 16:00:00


In [150]:
buyer_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,7872,7873,7874,7875,7876,7877,7878,7879,7880,7881
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~010373177cb75fac67,~01037460b8b2e770bb,~010374b5de700160df,~010376f8c47b978d93,~01037713e90f373b91,...,~01274b7db3f83e986c,~01274cff9a4becab2e,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
op_timezone,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC-05:00 Eastern Time (US & Canada),UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC (Coordinated Universal Time) Dublin, Edinb...",UTC-05:00 Eastern Time (US & Canada),UTC+10:00 Brisbane,UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,...,UTC-06:00 Central Time (US & Canada),"UTC+05:00 Islamabad, Karachi",UTC-05:00 Eastern Time (US & Canada),"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC-05:00 Eastern Time (US & Canada),"UTC+10:00 Sydney, Melbourne","UTC (Coordinated Universal Time) Dublin, Edinb...","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei",UTC-08:00 Pacific Time (US & Canada); Los Angeles,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels"
op_country,Sweden,United States,United States,United States,Netherlands,United Kingdom,United States,Australia,United States,United States,...,United States,Pakistan,United States,Italy,United States,Australia,United Kingdom,Singapore,United States,Denmark
op_city,Uppsala,Exeter,Fairfax,Newport Coast,Nijmegen,london,Elmwood Park,Gold Coast,Cumming,Arcata,...,chicago,Hyderabad,Oakland Gardens,Brusaporto,Vienna,Casula,London,Singapore,Los Angeles,Snekkersten
op_contract_date,"January 31, 2012","April 23, 2011","August 22, 2011","October 26, 2015","April 18, 2013","February 11, 2012","May 9, 2013","July 20, 2011","November 20, 2014","March 16, 2012",...,"January 14, 2012","December 27, 2009","September 20, 2012","March 19, 2008","February 10, 2010","August 18, 2012","November 4, 2012","March 15, 2012","August 15, 2008","June 10, 2012"


In [151]:
cand_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,146096,146097,146098,146099,146100,146101,146102,146103,146104,146105
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,...,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055,~0127566a84c383a055
cand_ciphertext,~01ca3b6ff3e632f6ab,~01eef64ef43a757c51,~01f1c24717cd7b9b1a,~014ea0e381dd57e4aa,~01a8d9c1ce117cd5f4,~01043970e768f1f1bf,~0125ca99f9886137c3,~01af9e3f87bd07bd37,~01b3b4f6c00e83d1ce,~0101eb1074369f013f,...,~012143fe423c737e05,~01b87e26e111ccf21f,~0110be1279e706cf30,~0197083dbd816c4f13,~011d5eddf1cd9067ca,~0160939c65766d58c0,~01f5d75e40b9b7aff1,~014e77cc7d6c4d44b7,~01e4bdce8490715995,~0130a751ae02b232d2
create_datetime,2012-02-19 07:04:06,2011-10-18 06:20:55,2015-05-06 00:19:14,2015-05-06 00:18:48,2015-05-06 00:17:30,2015-05-06 00:17:19,2015-05-06 00:15:21,2015-05-06 00:14:19,2015-05-06 00:12:55,2015-05-06 00:12:44,...,2012-06-11 05:55:30,2012-06-10 19:27:14,2012-06-10 15:06:39,2012-06-10 14:45:58,2012-06-10 14:40:19,2012-06-10 14:28:47,2012-06-10 14:05:08,2012-06-10 14:04:19,2012-06-10 14:03:22,2012-06-10 14:02:01


In [152]:
categ_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,7866,7867,7868,7869,7870,7871,7872,7873,7874,7875
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~010373177cb75fac67,~01037460b8b2e770bb,~010374b5de700160df,~010376f8c47b978d93,~01037713e90f373b91,...,~01274b7db3f83e986c,~01274cff9a4becab2e,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
category_group,"Web, Mobile & Software Dev",Writing,"Web, Mobile & Software Dev",Sales & Marketing,"Web, Mobile & Software Dev","Web, Mobile & Software Dev",Admin Support,"Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev",...,Sales & Marketing,"Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev",Data Science & Analytics,"Web, Mobile & Software Dev",Admin Support,"Web, Mobile & Software Dev",Writing


In [153]:
req_skills_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,5715,5716,5717,5718,5719,5720,5721,5722,5723,5724
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~01037460b8b2e770bb,~010374b5de700160df,~0103794b12d99e1b0e,~01037b2a4355a611ba,~01037b2c81e8cece68,...,~012749acb58c2698ae,~01274b7db3f83e986c,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
job_skill_set,"{adobedashillustrator, adobedashphotoshop}",{articledashwriting},{wordpress},"{internetdashresearch, leaddashgeneration, int...","{joomla, html, css, php}",{microsoftdashexcel},"{seo, joomla}","{jquery, seodashbacklinking, contentdashwritin...",{html},{adobedashflash},...,{microsoftdashaccessdashprogramming},{spreadsheets},"{adobedashfireworks, html, adobedashphotoshop,...",{html},"{php, wordpress}","{amazondashwebdashservices, apidashdevelopment...","{html, wordpress, css, php}","{internetdashresearch, webdashcontentdashmanag...","{svn, git, html, css}","{vba, microsoftdashexcel}"


In [154]:
profiles_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,88686,88687,88688,88689,88690,88691,88692,88693,88694,88695
dev_bill_rate,15,12,44.44,24.44,15,16.67,30,11.11,10,5.56,...,22.22,4,6.67,13.89,10,16.67,11.11,35,3,30
dev_blurb,"Over the more than 6 years, I have developed a...","I am a web application developer, having 2 yea...",Extensive expertise in clinical research publi...,I'm a WEB-developer with more than 5 years exp...,Specializes in developing Photoshop and After ...,,skype : orange_project_kh\nemail: contact@...,DTL Solutions is the software solutions provid...,I am an upcoming college graduate! I will rece...,1) Shopping Card Management System using C# a...,...,I'm a software engineer focused on backend and...,I am working as a officer at Checkpoint syst...,To enhance my journalistic prowess through onl...,I'm a highly motivated person with a strong ba...,"I like building cool web applications, and I o...",I configure and run the base station WiMAX on ...,I am looking to help in software solutions . w...,I have extensive experience using Python to so...,"Looking for work in proofreading, editing or c...",I have 8 Years of Profession Experience and i ...
dev_country,India,India,United States,Ukraine,Sri Lanka,Germany,Ukraine,Vietnam,United States,Bangladesh,...,Russia,Bangladesh,Philippines,Russia,India,Russia,Egypt,United States,India,India
dev_eng_skill,4,5,5,5,,5,4,4,5,,...,3,4,4,4,5,3,5,5,4,5
dev_portrait,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,,https://odesk-prod-portraits.s3.amazonaws.com/...,https://www.odesk.com/att/~~mOJyJwUvL-dOchJGOo...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...,https://odesk-prod-portraits.s3.amazonaws.com/...
dev_profile_title,LAMP developer with more than 6 years,Web Developer / Designer,"English/French Writer/Translator,Scientific Di...",PHP/Zend Framework/MySql/MongoDB/SPHINX/jQuery...,"After Effects, Photoshop - Scripting And Plugi...",Development of reliable and flexible software(...,"Fullstack web developer, e-commerce developing...",Small IT solution company provides Development...,Upbeat Enthusiastic Professional Writer,Bsc in Computer Science and Engineering,...,Software Developer,IT Professional,Statistician|News&Sports Writer|InDesign|Pagem...,Experienced developer and scientific assistant,An expert web application developer with Djang...,Software Developer,Ahmed Eshra,Python & ArcGIS Programmer,,Project Lead
dev_recno_ciphertext,~01cae3810dc6d2bbbc,~01d24ced467385214d,~01953fb4fa2ceb67dc,~01a9ab111f973ae055,~016dcec1087cba565c,~019e83a0dfa04c0e8b,~01863825efc778f975,~01614e82e3c5a0ccca,~0122260b02b0c56563,~01a3583a33f5146e8e,...,~01dfd63f31c5228d16,~018aaab6cfdc3fb96c,~01360017155d421ae3,~0169007d9fa447a06b,~014920c811cecebbc4,~0148cf1eca9907db89,~01baacbe6b9e3d8c30,~013659f86de224bdf9,~01b9ca1c9967daf519,~01a0cdfa47888e70f7
dev_timezone,"UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+02:00 Athens, Helsinki, Istanbul","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC+02:00 Eastern Europe,"UTC+07:00 Omsk, Novosibirsk",UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+06:00 Almaty, Dhaka",...,"UTC+07:00 Omsk, Novosibirsk","UTC+06:00 Almaty, Dhaka","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei","UTC+03:00 Moscow, St. Petersburg, Volg

### Delete multi-var variables that have been featurized (for the most part)

In [155]:
jobs_profiles_df.drop([
                       'assignment_info', 'assignments', \
                       'buyer', 'candidates', \
                       'op_tot_cand' \
                      ], axis=1, inplace=True)

# COMBINE JOB-LEVEL DATA SETS

In [156]:
jobs_profiles_df.T.head(38)

,0,1,2,3,4,5,6,7,8,9,...,7872,7873,7874,7875,7876,7877,7878,7879,7880,7881
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~010373177cb75fac67,~01037460b8b2e770bb,~010374b5de700160df,~010376f8c47b978d93,~01037713e90f373b91,...,~01274b7db3f83e986c,~01274cff9a4becab2e,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
job_type,Fixed,Fixed,Hourly,Hourly,Fixed,Fixed,Fixed,Hourly,Fixed,Hourly,...,Fixed,Hourly,Hourly,Hourly,Fixed,Fixed,Hourly,Hourly,Hourly,Hourly
op_title,Web designer project.,Article Writers - Fast and Reliable,WordPress Templates,Generate lead for special niche,Basic Joomla website,Create a dashboard to collect reviews and auto...,Update spread sheet,Joomla 1.5 site,Sidler-ng.com,Drupal Ecommerce Site Fix,...,Virtual Lead Generator Needed,Senior Python/Django/Bootstrap developer,"UI Design, Wireframing, Prototyping",Create html files using a template from text,Wordpress Expert,Scrapping data of amazon,"Wordpress theme fixes, HTML, CSS, PHP",Data Cleaning Specialist,w3c html/css + javascript django template desi...,Excel macro – VBA - Course
job_post_date,2012-02-19 07:01:16,2011-10-18 05:43:17,2015-05-05 23:56:37,2015-11-02 12:49:43,2013-06-12 06:30:16,2014-09-06 06:44:22,2014-01-13 19:55:28,2011-07-20 22:25:37,2014-11-20 11:54:33,2015-06-18 16:53:17,...,2012-01-13 19:24:47,2015-12-01 06:33:04,2012-10-21 08:17:20,2010-04-30 16:09:41,2010-06-03 05:22:26,2015-12-07 04:02:43,2012-11-04 13:38:15,2012-03-15 00:50:36,2010-08-15 07:17:57,2012-06-10 13:57:13
op_description_stripped,We need some deisgns for our website. Its exac...,Each writer will be give a set of keywords. Mo...,Post archive page and post page designed to ma...,"Hi, We are a company with a special niche and ...","Hi, We're looking for someone with Joomla, HTM...",Morning I'm looking for a developer to create ...,Update images url for 2000 products,"Hi, Im wanting to have a joomla template chang...",Modernize,Need help getting my Drupal store back up and ...,...,This job requires you to have a fast internet ...,- Job description I need a senior django devel...,Surpass the Limit is a social platform where u...,Create the speaker pages for a conference. Sta...,"Hi, we need a few themes customized for Wordpr...","Hi Mubbashar, as discussed, i need to scrape t...",We need a web developer to implement a few rem...,We run a travel destination portal. On the sit...,We have an immediate need for an html/css code...,I need an Excel Macro (VBA) course for my web ...


In [157]:
df_list = [ \
jobs_profiles_df,
buyer_df, \
categ_df, \
req_skills_df, \
profiles_df
]

for df in df_list:
    for col in list(df.columns):
        print col
    print

ciphertext
job_type
op_title
job_post_date
op_description_stripped

ciphertext
op_timezone
op_country
op_city
op_contract_date

ciphertext
category_group

ciphertext
job_skill_set

dev_bill_rate
dev_blurb
dev_country
dev_eng_skill
dev_portrait
dev_profile_title
dev_recno_ciphertext
dev_timezone
education
agency_affl
group_affl
has_portrait
highest_degree
has_portfolio
took_tests
skills_string



In [158]:
job_level_df = pd.merge(jobs_profiles_df, buyer_df, how='left', on='ciphertext')

In [159]:
job_level_df = pd.merge(job_level_df, categ_df, how='left', on='ciphertext')

In [160]:
job_level_df = pd.merge(job_level_df, req_skills_df, how='left', on='ciphertext')

In [161]:
job_level_df.T.head(25)

,0,1,2,3,4,5,6,7,8,9,...,7872,7873,7874,7875,7876,7877,7878,7879,7880,7881
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~010373177cb75fac67,~01037460b8b2e770bb,~010374b5de700160df,~010376f8c47b978d93,~01037713e90f373b91,...,~01274b7db3f83e986c,~01274cff9a4becab2e,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~012750a2a32232cd0d,~01275285137d94fcb4,~012753e9dd081c4643,~0127547bbab346404b,~01275667fa5be0de44,~0127566a84c383a055
job_type,Fixed,Fixed,Hourly,Hourly,Fixed,Fixed,Fixed,Hourly,Fixed,Hourly,...,Fixed,Hourly,Hourly,Hourly,Fixed,Fixed,Hourly,Hourly,Hourly,Hourly
op_title,Web designer project.,Article Writers - Fast and Reliable,WordPress Templates,Generate lead for special niche,Basic Joomla website,Create a dashboard to collect reviews and auto...,Update spread sheet,Joomla 1.5 site,Sidler-ng.com,Drupal Ecommerce Site Fix,...,Virtual Lead Generator Needed,Senior Python/Django/Bootstrap developer,"UI Design, Wireframing, Prototyping",Create html files using a template from text,Wordpress Expert,Scrapping data of amazon,"Wordpress theme fixes, HTML, CSS, PHP",Data Cleaning Specialist,w3c html/css + javascript django template desi...,Excel macro – VBA - Course
job_post_date,2012-02-19 07:01:16,2011-10-18 05:43:17,2015-05-05 23:56:37,2015-11-02 12:49:43,2013-06-12 06:30:16,2014-09-06 06:44:22,2014-01-13 19:55:28,2011-07-20 22:25:37,2014-11-20 11:54:33,2015-06-18 16:53:17,...,2012-01-13 19:24:47,2015-12-01 06:33:04,2012-10-21 08:17:20,2010-04-30 16:09:41,2010-06-03 05:22:26,2015-12-07 04:02:43,2012-11-04 13:38:15,2012-03-15 00:50:36,2010-08-15 07:17:57,2012-06-10 13:57:13
op_description_stripped,We need some deisgns for our website. Its exac...,Each writer will be give a set of keywords. Mo...,Post archive page and post page designed to ma...,"Hi, We are a company with a special niche and ...","Hi, We're looking for someone with Joomla, HTM...",Morning I'm looking for a developer to create ...,Update images url for 2000 products,"Hi, Im wanting to have a joomla template chang...",Modernize,Need help getting my Drupal store back up and ...,...,This job requires you to have a fast internet ...,- Job description I need a senior django devel...,Surpass the Limit is a social platform where u...,Create the speaker pages for a conference. Sta...,"Hi, we need a few themes customized for Wordpr...","Hi Mubbashar, as discussed, i need to scrape t...",We need a web developer to implement a few rem...,We run a travel destination portal. On the sit...,We have an immediate need for an html/css code...,I need an Excel Macro (VBA) course for my web ...
op_timezone,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC-05:00 Eastern Time (US & Canada),UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC (Coordinated Universal Time) Dublin, Edinb...",UTC-05:00 Eastern Time (US & Canada),UTC+10:00 Brisbane,UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,...,UTC-06:00 Central Time (US & Canada),"UTC+05:00 Islamabad, Karachi",UTC-05:00 Eastern Time (US & Canada),"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC-05:00 Eastern Time (US & Canada),"UTC+10:00 Sydney, Melbourne","UTC (Coordinated Universal Time) Dublin, Edinb...","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei",UTC-08:00 Pacific Time (US & Canada); Los Angeles,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels"
op_country,Sweden,United States,United States,United States,Netherlands,United Kingdom,United States,Australia,United States,United States,...,United States,Pakistan,United States,Italy,United States,Australia,United Kingdom,Singapore,United States,Denmark
op_city,Uppsala,Exeter,Fairfax,Newport Coast,Nijmegen,london,Elmwood Park,Gold Coast,Cumming,Arcata,...,chicago,Hyderabad,Oakland Gardens,Brusaporto,Vienna,Casula,London,Singapore,Los Angeles,Snekkersten
op_contract_date

In [162]:
job_level_df.drop(['op_city', 'op_contract_date'], axis=1, inplace=True)

# CREATE CANDIDATE-LEVEL DATA SET (JOB-PERSON TUPLES)

In [163]:
# DUPE CHECK
# jj = profiles_df.duplicated('dev_recno_ciphertext')
# any(jj = True)

In [164]:
def string_to_set(str):
    return set(str.split())
profiles_df['skills_set'] = profiles_df['skills_string'].apply(string_to_set)

In [165]:
profiles_df.drop(['dev_portrait', 'education', 'skills_string'], axis=1, inplace=True)

In [166]:
profiles_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,88686,88687,88688,88689,88690,88691,88692,88693,88694,88695
dev_bill_rate,15,12,44.44,24.44,15,16.67,30,11.11,10,5.56,...,22.22,4,6.67,13.89,10,16.67,11.11,35,3,30
dev_blurb,"Over the more than 6 years, I have developed a...","I am a web application developer, having 2 yea...",Extensive expertise in clinical research publi...,I'm a WEB-developer with more than 5 years exp...,Specializes in developing Photoshop and After ...,,skype : orange_project_kh\nemail: contact@...,DTL Solutions is the software solutions provid...,I am an upcoming college graduate! I will rece...,1) Shopping Card Management System using C# a...,...,I'm a software engineer focused on backend and...,I am working as a officer at Checkpoint syst...,To enhance my journalistic prowess through onl...,I'm a highly motivated person with a strong ba...,"I like building cool web applications, and I o...",I configure and run the base station WiMAX on ...,I am looking to help in software solutions . w...,I have extensive experience using Python to so...,"Looking for work in proofreading, editing or c...",I have 8 Years of Profession Experience and i ...
dev_country,India,India,United States,Ukraine,Sri Lanka,Germany,Ukraine,Vietnam,United States,Bangladesh,...,Russia,Bangladesh,Philippines,Russia,India,Russia,Egypt,United States,India,India
dev_eng_skill,4,5,5,5,,5,4,4,5,,...,3,4,4,4,5,3,5,5,4,5
dev_profile_title,LAMP developer with more than 6 years,Web Developer / Designer,"English/French Writer/Translator,Scientific Di...",PHP/Zend Framework/MySql/MongoDB/SPHINX/jQuery...,"After Effects, Photoshop - Scripting And Plugi...",Development of reliable and flexible software(...,"Fullstack web developer, e-commerce developing...",Small IT solution company provides Development...,Upbeat Enthusiastic Professional Writer,Bsc in Computer Science and Engineering,...,Software Developer,IT Professional,Statistician|News&Sports Writer|InDesign|Pagem...,Experienced developer and scientific assistant,An expert web application developer with Djang...,Software Developer,Ahmed Eshra,Python & ArcGIS Programmer,,Project Lead
dev_recno_ciphertext,~01cae3810dc6d2bbbc,~01d24ced467385214d,~01953fb4fa2ceb67dc,~01a9ab111f973ae055,~016dcec1087cba565c,~019e83a0dfa04c0e8b,~01863825efc778f975,~01614e82e3c5a0ccca,~0122260b02b0c56563,~01a3583a33f5146e8e,...,~01dfd63f31c5228d16,~018aaab6cfdc3fb96c,~01360017155d421ae3,~0169007d9fa447a06b,~014920c811cecebbc4,~0148cf1eca9907db89,~01baacbe6b9e3d8c30,~013659f86de224bdf9,~01b9ca1c9967daf519,~01a0cdfa47888e70f7
dev_timezone,"UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+02:00 Athens, Helsinki, Istanbul","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC+02:00 Eastern Europe,"UTC+07:00 Omsk, Novosibirsk",UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+06:00 Almaty, Dhaka",...,"UTC+07:00 Omsk, Novosibirsk","UTC+06:00 Almaty, Dhaka","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei","UTC+03:00 Moscow, St. Petersburg, Volgograd",UTC (Coordinated Universal Time),"UTC+04:00 Abu Dhabi, Muscat, Tbilisi, Kazan",UTC+02:00 Cairo,UTC-07:00 Mountain Time (US & Canada),"UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi","UTC+05:30 Mumbai, Kolkata, Chennai, New Delhi"
agency_affl,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
group_affl,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
has_portrait,True,True,True,False,True,False,True,True,True,True,...,True,True,True,True,False,True,True,True,True,True


In [167]:
for col in list(profiles_df.columns):
    print col

dev_bill_rate
dev_blurb
dev_country
dev_eng_skill
dev_profile_title
dev_recno_ciphertext
dev_timezone
agency_affl
group_affl
has_portrait
highest_degree
has_portfolio
took_tests
skills_set


In [168]:
cand_level_df = pd.merge( \
                           cand_df, assi_df, how='left', \
                           left_on=['ciphertext', 'cand_ciphertext'], \
                           right_on=['ciphertext', 'ciphertext_developer_recno'], \
                           indicator=True \
                          )

In [169]:
cand_level_df['got_assignment'] = (cand_level_df['_merge'] == 'both').apply(lambda x: 1 if x else 0)

In [170]:
cand_level_df.drop(['_merge'], axis=1, inplace=True)

### GRAFT PERSON-LEVEL and JOB-LEVEL DATA SETS TO CANDIDATE=(PERSON, JOB)-LEVEL DATA SET

In [171]:
cand_level_df = pd.merge( \
                           cand_level_df, profiles_df, how='inner', \
                           left_on='cand_ciphertext', \
                           right_on='dev_recno_ciphertext', \
                          )

In [172]:
cand_level_df = pd.merge( \
                           job_level_df, cand_level_df, how='inner', \
                           on='ciphertext'
                          )

In [173]:
cand_level_df.T.head(38)

,0,1,2,3,4,5,6,7,8,9,...,20148,20149,20150,20151,20152,20153,20154,20155,20156,20157
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~01037313aee93f572b,~01037313aee93f572b,~01037313aee93f572b,...,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~012753e9dd081c4643,~012753e9dd081c4643,~012753e9dd081c4643,~0127547bbab346404b,~0127566a84c383a055
job_type,Fixed,Fixed,Hourly,Hourly,Hourly,Hourly,Fixed,Fixed,Fixed,Fixed,...,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly
op_title,Web designer project.,Article Writers - Fast and Reliable,WordPress Templates,WordPress Templates,WordPress Templates,Generate lead for special niche,Basic Joomla website,Basic Joomla website,Basic Joomla website,Basic Joomla website,...,"UI Design, Wireframing, Prototyping",Create html files using a template from text,Create html files using a template from text,Create html files using a template from text,Create html files using a template from text,"Wordpress theme fixes, HTML, CSS, PHP","Wordpress theme fixes, HTML, CSS, PHP","Wordpress theme fixes, HTML, CSS, PHP",Data Cleaning Specialist,Excel macro – VBA - Course
job_post_date,2012-02-19 07:01:16,2011-10-18 05:43:17,2015-05-05 23:56:37,2015-05-05 23:56:37,2015-05-05 23:56:37,2015-11-02 12:49:43,2013-06-12 06:30:16,2013-06-12 06:30:16,2013-06-12 06:30:16,2013-06-12 06:30:16,...,2012-10-21 08:17:20,2010-04-30 16:09:41,2010-04-30 16:09:41,2010-04-30 16:09:41,2010-04-30 16:09:41,2012-11-04 13:38:15,2012-11-04 13:38:15,2012-11-04 13:38:15,2012-03-15 00:50:36,2012-06-10 13:57:13
op_description_stripped,We need some deisgns for our website. Its exac...,Each writer will be give a set of keywords. Mo...,Post archive page and post page designed to ma...,Post archive page and post page designed to ma...,Post archive page and post page designed to ma...,"Hi, We are a company with a special niche and ...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...",...,Surpass the Limit is a social platform where u...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,We need a web developer to implement a few rem...,We need a web developer to implement a few rem...,We need a web developer to implement a few rem...,We run a travel destination portal. On the sit...,I need an Excel Macro (VBA) course for my web ...
op_timezone,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",UTC-05:00 Eastern Time (US & Canada),UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,UTC-08:00 Pacific Time (US & Canada); Tijuana,"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels",...,UTC-05:00 Eastern Time (US & Canada),"UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels","UTC (Coordinated Universal Time) Dublin, Edinb...","UTC (Coordinated Universal Time) Dublin, Edinb...","UTC (Coordinated Universal Time) Dublin, Edinb...","UTC+08:00 Hong Kong SAR, Perth, Singapore, Taipei","UTC+01:00 Berlin, Stockholm, Rome, Bern, Brussels"
op_country,Sweden,United States,United States,United States,United States,United States,Netherlands,Netherlands,Netherlands,Netherlands,...,United States,Italy,Italy,Italy,Italy,United Kingdom,United Kingdom,United Kingdom,Singapore,Denmark
category_group,"W

In [174]:
for col in list(cand_level_df.columns):
    print col


ciphertext
job_type
op_title
job_post_date
op_description_stripped
op_timezone
op_country
category_group
job_skill_set
cand_ciphertext
create_datetime
ciphertext_developer_recno
start_datetime
end_datetime
got_assignment
dev_bill_rate
dev_blurb
dev_country
dev_eng_skill
dev_profile_title
dev_recno_ciphertext
dev_timezone
agency_affl
group_affl
has_portrait
highest_degree
has_portfolio
took_tests
skills_set


### CREATE DERIVED VARIABLES FOR CANDIDATE-LEVEL DATA SET

In [175]:
skl_tuples = list()
for index, skl_tuple in \
  enumerate(cand_level_df[['ciphertext', 'job_skill_set', 'skills_set']].itertuples(index=False)):
    profile_id = skl_tuple[0]
    job_skill_set = skl_tuple[1]
    provider_skill_set = skl_tuple[2]

    if isinstance(job_skill_set, set):        
        size_job_skill_set = len(job_skill_set)
        counter = 0
        if isinstance(provider_skill_set, set):
            for skill in job_skill_set:
                if skill in provider_skill_set:
                    counter += 1
            skill_alignment = counter / float(size_job_skill_set)
        else:
            skill_alignment = 0
    else:
        skill_alignment = np.nan
    
    skl_tuples.append( (profile_id, skill_alignment) )


In [176]:
print len(skl_tuples)

20158


In [177]:
skl_df = pd.DataFrame(skl_tuples)

#### `dev_blurb`

In [184]:
cand_level_df['dev_blurb_stripped'] = cand_level_df['dev_blurb'].apply(stripped)

### Drop variables that are hard to use, superseded by others (e.g., date variables), or redundant

In [179]:
cand_level_df.drop([ \
                  'dev_blurb', \
                  'cand_ciphertext', \
                  'op_timezone', \
                  'dev_timezone', \
                  'create_datetime', \
                  'start_datetime', \
                  'end_datetime', \
                  'ciphertext_developer_recno' \
                  'job_skill_set', \
                  'skills_set' \
                   ], axis=1, inplace=True)

In [197]:
cand_level_df.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,20148,20149,20150,20151,20152,20153,20154,20155,20156,20157
ciphertext,~01036ab99176fc9ca7,~01036ecca7fbdb9d47,~0103703f207be9f938,~0103703f207be9f938,~0103703f207be9f938,~01037164e7e1613e2f,~01037313aee93f572b,~01037313aee93f572b,~01037313aee93f572b,~01037313aee93f572b,...,~01274d1cb929fb7d32,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~0127500b1b70ff31b2,~012753e9dd081c4643,~012753e9dd081c4643,~012753e9dd081c4643,~0127547bbab346404b,~0127566a84c383a055
job_type,Fixed,Fixed,Hourly,Hourly,Hourly,Hourly,Fixed,Fixed,Fixed,Fixed,...,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly,Hourly
op_title,Web designer project.,Article Writers - Fast and Reliable,WordPress Templates,WordPress Templates,WordPress Templates,Generate lead for special niche,Basic Joomla website,Basic Joomla website,Basic Joomla website,Basic Joomla website,...,"UI Design, Wireframing, Prototyping",Create html files using a template from text,Create html files using a template from text,Create html files using a template from text,Create html files using a template from text,"Wordpress theme fixes, HTML, CSS, PHP","Wordpress theme fixes, HTML, CSS, PHP","Wordpress theme fixes, HTML, CSS, PHP",Data Cleaning Specialist,Excel macro – VBA - Course
job_post_date,2012-02-19 07:01:16,2011-10-18 05:43:17,2015-05-05 23:56:37,2015-05-05 23:56:37,2015-05-05 23:56:37,2015-11-02 12:49:43,2013-06-12 06:30:16,2013-06-12 06:30:16,2013-06-12 06:30:16,2013-06-12 06:30:16,...,2012-10-21 08:17:20,2010-04-30 16:09:41,2010-04-30 16:09:41,2010-04-30 16:09:41,2010-04-30 16:09:41,2012-11-04 13:38:15,2012-11-04 13:38:15,2012-11-04 13:38:15,2012-03-15 00:50:36,2012-06-10 13:57:13
op_description_stripped,We need some deisgns for our website. Its exac...,Each writer will be give a set of keywords. Mo...,Post archive page and post page designed to ma...,Post archive page and post page designed to ma...,Post archive page and post page designed to ma...,"Hi, We are a company with a special niche and ...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...","Hi, We're looking for someone with Joomla, HTM...",...,Surpass the Limit is a social platform where u...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,Create the speaker pages for a conference. Sta...,We need a web developer to implement a few rem...,We need a web developer to implement a few rem...,We need a web developer to implement a few rem...,We run a travel destination portal. On the sit...,I need an Excel Macro (VBA) course for my web ...
op_country,Sweden,United States,United States,United States,United States,United States,Netherlands,Netherlands,Netherlands,Netherlands,...,United States,Italy,Italy,Italy,Italy,United Kingdom,United Kingdom,United Kingdom,Singapore,Denmark
category_group,"Web, Mobile & Software Dev",Writing,"Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev",Sales & Marketing,"Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev",...,"Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev","Web, Mobile & Software Dev",Admin Support,Writing
got_assignment,1,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
dev_bill_rate,20,10,7,15,10,7,25,11.11,22.22,8,...,30,15.56,25,11.11,5,16.67,25,15,22.22,16.67
dev_country,Bangladesh,Bangladesh,Pakistan,India,India,Philippines,India,India,India,Canada,...,Poland,Bangladesh,Moldova,India,Pakistan,United States,Bangladesh,India,Philippines,India


### `dev_country`

In [231]:
print cand_level_df[cand_level_df['op_title'] == 'Photoshop script to do face swap']

Empty DataFrame
Columns: [ciphertext, job_type, op_title, job_post_date, op_description_stripped, op_country, category_group, got_assignment, dev_bill_rate, dev_country, dev_eng_skill, dev_profile_title, dev_recno_ciphertext, agency_affl, group_affl, has_portrait, highest_degree, has_portfolio, took_tests, dev_blurb_stripped, usa_or_canada]
Index: []

[0 rows x 21 columns]


In [204]:
# cand_level_df.dev_country.value_counts()

In [201]:
def usa_or_canada(str):
    return 1 if str == 'United States' or str =='Canada' else 0
cand_level_df['usa_or_canada'] = cand_level_df['dev_country'].apply(usa_or_canada)

## Counting previous jobs completed

In [232]:
jobs_df.head(20)

,dev_recno_ciphertext,as_opening_title,as_from_full,as_to_full,as_rate,as_total_hours_precise,as_total_charge,as_job_type,end_date_str,end_date
0,~01cae3810dc6d2bbbc,Development of Firovia client/associate projec...,,01/09/2013,,0,,Fixed,01/09/2013,2013-01-09
1,~01a9ab111f973ae055,Zend Senior Developer + Junior Developer for b...,,Present,$22.22,614,13642.99,Hourly,05/04/2016,2016-05-04
2,~01a9ab111f973ae055,Top notch PHP / Zend Developer (experienced),,02/07/2013,$16.67,1195.66666666667,19932.02,Hourly,02/07/2013,2013-02-07
3,~01a9ab111f973ae055,Process Bounced member records,12/24/2012,12/24/2012,$0.00,0,65.56,Fixed,12/24/2012,2012-12-24
4,~016dcec1087cba565c,Photoshop script to do face swap,03/15/2016,04/12/2016,$0.00,0,200,Fixed,04/12/2016,2016-04-12
5,~019e83a0dfa04c0e8b,Sr. Oracle PL/SQL Developer,,09/30/2011,,1.1666666666667,,Hourly,09/30/2011,2011-09-30
6,~019e83a0dfa04c0e8b,windowless / chromeless browser project,,08/19/2010,$11.11,10,111.1,Hourly,08/19/2010,2010-08-19
7,~019e83a0dfa04c0e8b,Dropdown window on key press - C++,,01/23/2013,$0.00,0,21.67,Fixed,01/23/2013,2013-01-23
8,~01863825efc778f975,Web programmer/developer,04/04/2016,04/08/2016,$0.00,0,110,Fixed,04/08/2016,2016-04-08
9,~01863825efc778f975,website speed,03/04/2016,03/11/2016,$0.00,0,200,Fixed,03/11/2016,2016-03-11


In [207]:
cand_level_df['dev_recno_ciphertext'].head()

0    ~01ca3b6ff3e632f6ab
1    ~01eef64ef43a757c51
2    ~0140a8c1b287ababb9
3    ~011f743ae5239f3844
4    ~01d450ed168acd100f
Name: dev_recno_ciphertext, dtype: object

In [211]:
job_count_df = pd.merge( \
                           cand_level_df[['dev_recno_ciphertext', 'job_post_date']], jobs_df, how='left', \
                           on='dev_recno_ciphertext', \
                           indicator=True \
                          )

In [212]:
job_count_df.head()

,dev_recno_ciphertext,job_post_date,as_opening_title,as_from_full,as_to_full,as_rate,as_total_hours_precise,as_total_charge,as_job_type,_merge
0,~01ca3b6ff3e632f6ab,2012-02-19 07:01:16,AngularJS/Laravel Developer,,Present,$18.00,103.666666666667,1866,Hourly,both
1,~01ca3b6ff3e632f6ab,2012-02-19 07:01:16,Stanford Research Team seeks Front End Develop...,,10/06/2015,$20.00,24.5,490,Hourly,both
2,~01ca3b6ff3e632f6ab,2012-02-19 07:01:16,The best Wordpress/HTML/CSS Developer in the P...,08/28/2015,09/22/2015,$20.00,1.66666666666667,33.33,Hourly,both
3,~01ca3b6ff3e632f6ab,2012-02-19 07:01:16,Developer for Wordpress Multisite and more,,08/27/2015,,732.5,,Hourly,both
4,~01ca3b6ff3e632f6ab,2012-02-19 07:01:16,Stanford Research Team seeks Front End Develop...,,08/20/2015,$20.00,3,60,Hourly,both


In [223]:
todays_date = datetime.datetime.now().strftime('%m/%d/%Y')
job_count_df['end_date_str'] = job_count_df['as_to_full'].replace({'Present': todays_date})
job_count_df['end_date'] = pd.to_datetime(job_count_df['end_date_str'])


In [224]:
print job_count_df['end_date'].head()

0   2016-05-04
1   2015-10-06
2   2015-09-22
3   2015-08-27
4   2015-08-20
Name: end_date, dtype: datetime64[ns]


In [225]:
job_count_df['completed_before_new_job'] = (job_count_df['end_date']< job_count_df['job_post_date']) 


In [228]:
# job_count_df[['completed_before_new_job', 'end_date', 'job_post_date']].head(50)

# GLMM in Python!

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.MixedLM.from_formula("Weight ~ Time", data, groups=data["Pig"])
result = model.fit()
print result.summary()